In [2]:
# -*- coding: utf-8 -*-
"""
LangGraph: 3 inputs -> 3 poster prompts (LLM-decided)
- Inputs: festival_info (dict), keywords (str|list), planning_intent (str), aspect (str, default "4:5")
- LLM이 스타일/팔레트/모티프/조명/레이아웃/타이포/슬로건/네거티브/품질을 자동 판단
- Output:
  - prompts (dict: photographic_cinematic / minimal_graphic / illustration_papercut)
  - formatted (str): 예시와 동일한 구분선 출력 문자열
  - design_plan (LLM이 만든 구조화 JSON)
"""

from __future__ import annotations
from typing import Dict, List, TypedDict, Union, Optional
import os, json, re, time

# LangChain / LangGraph
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langgraph.graph import StateGraph, START, END

# Pydantic (LangChain 호환 v1 API)
from langchain_core.pydantic_v1 import BaseModel, Field


# =========================
# 0) Pydantic schema (LLM)
# =========================
class Slogans(BaseModel):
    photographic: str = Field(..., description="Photographic/Cinematic 톤의 슬로건")
    minimal: str = Field(..., description="미니멀/타이포 중심 슬로건")
    illustration: str = Field(..., description="일러스트/페이퍼컷 톤의 슬로건")

class DesignPlan(BaseModel):
    palette: List[str]
    textures: List[str]
    lighting: str
    lighting_hint: str
    motifs: List[str]
    layout: str
    typography: str
    slogans: Slogans
    quality: List[str]
    negatives: List[str]


# ======================
# 1) Graph state schema
# ======================
class BuildState(TypedDict, total=False):
    festival_info: Dict[str, str]
    keywords: Union[str, List[str]]
    planning_intent: str
    aspect: str
    design_plan: DesignPlan
    prompts: Dict[str, str]
    formatted: str


# =========================
# 2) Utility: Format output
# =========================
LABELS = [
    ("Photographic / Cinematic", "photographic_cinematic"),
    ("Minimal Graphic", "minimal_graphic"),
    ("Illustration / Papercut", "illustration_papercut"),
]

def format_prompts_exact(prompts: Dict[str, str]) -> str:
    chunks = []
    for label, key in LABELS:
        block = (prompts.get(key) or "").strip()
        header = f"{'='*20} {label} {'='*20}"
        chunks.append(header + "\n" + block + "\n")
    return "\n".join(chunks).rstrip()

def validate_required_fields(prompts: Dict[str, str]) -> None:
    reqs = ["H1:", "H2:", "H3:", "Caption(하단):"]
    for label, key in LABELS:
        text = prompts.get(key, "") or ""
        missing = [r for r in reqs if r not in text]
        if missing:
            raise AssertionError(f"[{label}] 누락 필드: {missing}")


# =========================
# 3) Nodes (LangGraph)
# =========================
def normalize_node(state: BuildState) -> BuildState:
    # aspect 기본값
    aspect = state.get("aspect") or "4:5"

    # keywords 문자열 -> 리스트 정규화(LLM에는 그냥 전달해도 되지만 추후 활용 대비)
    kws = state.get("keywords", [])
    if isinstance(kws, str):
        kws = [p.strip() for p in kws.replace("，", ",").split(",") if p.strip()]

    out: BuildState = dict(state)
    out["aspect"] = aspect
    out["keywords"] = kws or state.get("keywords")
    return out


def design_plan_node(state: BuildState) -> BuildState:
    """LLM이 전체 디자인 계획을 구조화해서 반환."""
    festival_info = state["festival_info"]
    keywords = state["keywords"]
    planning_intent = state["planning_intent"]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.6)

    sys_tmpl = SystemMessagePromptTemplate.from_template(
        "당신은 수상 경력의 포스터 아트디렉터입니다. "
        "입력(축제정보, 키워드, 기획의도)만 보고, 포스터 디자인을 위한 결정을 **스스로 판단**하여 "
        "아래 스키마에 맞는 JSON을 한국어로 출력하세요. 오직 JSON만 반환하세요."
    )
    human_tmpl = HumanMessagePromptTemplate.from_template(
        """다음 입력을 참고하여 스키마에 맞게 작성하세요.
[festival_info]: {festival_info}
[keywords]: {keywords}
[planning_intent]: {planning_intent}

[SCHEMA]
- palette: string[]
- textures: string[]
- lighting: string
- lighting_hint: string
- motifs: string[]
- layout: string
- typography: string
- slogans: {{ photographic: string, minimal: string, illustration: string }}
- quality: string[]
- negatives: string[]"""
    )
    prompt = ChatPromptTemplate.from_messages([sys_tmpl, human_tmpl])

    # LLM structured output (Pydantic)
    structured_llm = llm.with_structured_output(DesignPlan)
    plan: DesignPlan = structured_llm.invoke(
        prompt.format_messages(
            festival_info=festival_info,
            keywords=keywords,
            planning_intent=planning_intent,
        )
    )

    out: BuildState = dict(state)
    out["design_plan"] = plan
    return out


def build_prompts_node(state: BuildState) -> BuildState:
    """디자인 플랜 -> 3가지 프롬프트(정확 텍스트 포함)."""
    plan: DesignPlan = state["design_plan"]
    aspect: str = state["aspect"]

    title = state["festival_info"].get("행사명", "").strip()
    period = state["festival_info"].get("행사 기간", "").strip()
    place = state["festival_info"].get("장소", "").strip()
    organizer = state["festival_info"].get("주최/주관", "").strip()

    exact_text = (
        f"H1: **{title}**\n"
        f"H2: **{period}**\n"
        f"H3: **{place}**\n"
        f"Caption(하단): **주최/주관: {organizer}**"
    )

    palette = ", ".join(plan.palette)
    textures = ", ".join(plan.textures)
    motifs = ", ".join(plan.motifs)
    lighting = plan.lighting
    lighting_hint = plan.lighting_hint
    layout = plan.layout
    typo = plan.typography
    qual = ", ".join(plan.quality)
    neg = ", ".join(plan.negatives)

    p1 = f"""고급 인쇄 포스터, Aspect: {aspect}, 최소 2160x2700 px, **정확한 한국어 타이포그래피**.
주제/비주얼 요소: {motifs}. 텍스처: {textures}. 조명: {lighting} ({lighting_hint}).
팔레트: {palette}.
레이아웃: {layout}.
타입: {typo}. **& 기호는 반드시 ‘&’ 그대로 사용**.
**포스터에 반드시 들어갈 정확한 텍스트(오탈자 없이 그대로):**
{exact_text}
슬로건(선택): **{plan.slogans.photographic}**
품질: {qual}.
Negative: {neg}."""

    p2 = f"""미니멀 그래픽 포스터, Aspect: {aspect}, 최소 2160x2700 px, **정확한 한국어 텍스트**.
스타일: 팔레트({palette})에서 2–3색 중심, 여백과 대비를 살린 클린 레이아웃.
레이아웃: {layout}.
타입: {typo}. **&는 기호 그대로**.
**정확히 표기할 텍스트:**
{exact_text}
슬로건(선택): **{plan.slogans.minimal}**
품질: {qual}.
Negative: {neg}."""

    p3 = f"""일러스트/페이퍼컷 감성 포스터, Aspect: {aspect}, 최소 2160x2700 px, **정확한 한국어 텍스트**.
무드: 팔레트({palette}), 텍스처({textures}).
구성: {motifs} 요소가 **보행/체류 동선**처럼 이어지도록. {lighting_hint}.
레이아웃: {layout}.
타입: {typo}. **&는 기호 그대로**.
**정확히 표기할 텍스트:**
{exact_text}
슬로건(선택): **{plan.slogans.illustration}**
품질: {qual}.
Negative: {neg}."""

    prompts = {
        "photographic_cinematic": p1.strip(),
        "minimal_graphic": p2.strip(),
        "illustration_papercut": p3.strip(),
    }

    out: BuildState = dict(state)
    out["prompts"] = prompts
    return out


def format_node(state: BuildState) -> BuildState:
    prompts = state["prompts"]
    validate_required_fields(prompts)
    formatted = format_prompts_exact(prompts)
    out: BuildState = dict(state)
    out["formatted"] = formatted
    return out


# =========================
# 4) Build the LangGraph
# =========================
def build_graph():
    graph = StateGraph(BuildState)
    graph.add_node("normalize", normalize_node)
    graph.add_node("design_plan", design_plan_node)
    graph.add_node("build_prompts", build_prompts_node)
    graph.add_node("format", format_node)

    graph.add_edge(START, "normalize")
    graph.add_edge("normalize", "design_plan")
    graph.add_edge("design_plan", "build_prompts")
    graph.add_edge("build_prompts", "format")
    graph.add_edge("format", END)
    return graph.compile()


# =========================
# 5) Example run (main)
# =========================
if __name__ == "__main__":
    # ✏️ 여기 4개만 바꿔서 재사용
    festival_info = {
        "행사명": "2024년 양림 & 크리스마스 문화축제",
        "행사 기간": "2024. 12. 2.(월) ~ 12. 31.(화)",
        "장소": "양림동 일원 (양림오거리 ~ 선교묘역)",
        "주최/주관": "광주광역시 남구 / 양림 & 크리스마스 문화축제추진위원회",
    }
    keywords = "크리스마스, 연말, 낭만"  # or ["크리스마스","연말","낭만"]
    planning_intent = (
        "양림동의 근대 역사와 골목 풍경을 배경으로, 12월 한 달간 시민 누구나 일상에서 ‘연말의 낭만’을 회복하는 겨울 문화축제를 엽니다. "
        "크리스마스 감성의 야간 경관과 골목 산책, 버스킹·합창, 따뜻한 마켓과 나눔 프로그램을 결합해 지역 상권과 공동체가 함께 빛나는 장을 만듭니다. "
        "양림오거리–선교묘역을 잇는 안전한 보행 동선 위에 포토존과 휴식 공간을 배치해, 걷고 머무르며 추억을 남기는 도심 산책형 축제를 지향합니다."
    )
    aspect = "4:5"  # 원하는 경우 "2:3"

    app = build_graph()
    result: BuildState = app.invoke({
        "festival_info": festival_info,
        "keywords": keywords,
        "planning_intent": planning_intent,
        "aspect": aspect,
    })

    # 콘솔 출력 (예시와 동일한 포맷)
    print(result["formatted"])

    # 저장(옵션)
    ts = time.strftime("%Y%m%d_%H%M%S")
    base = f"./poster_prompts_{ts}"
    with open(base + "_design_plan.json", "w", encoding="utf-8") as f:
        # DesignPlan -> dict
        plan_dict = json.loads(result["design_plan"].json())
        json.dump(plan_dict, f, ensure_ascii=False, indent=2)

    with open(base + "_prompts.json", "w", encoding="utf-8") as f:
        json.dump(result["prompts"], f, ensure_ascii=False, indent=2)

    with open(base + "_prompts.txt", "w", encoding="utf-8") as f:
        f.write(result["formatted"] + "\n")

    print("\nSaved:")
    print(base + "_design_plan.json")
    print(base + "_prompts.json")
    print(base + "_prompts.txt")


c:\workspace\uv_festival\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
c:\workspace\uv_festival\.venv\Lib\site-packages\langchain_openai\chat_models\base.py:1900: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="fun

==================== Photographic / Cinematic ====================
고급 인쇄 포스터, Aspect: 4:5, 최소 2160x2700 px, **정확한 한국어 타이포그래피**.
주제/비주얼 요소: 크리스마스 트리, 눈송이, 양림동 골목, 버스킹 공연. 텍스처: 부드러운 천, 나무, 눈, 빛나는 장식. 조명: 부드럽고 따뜻한 조명 (크리스마스 트리와 길거리 조명을 활용하여 아늑한 분위기를 연출).
팔레트: 따뜻한 빨강, 녹색, 금색, 흰색.
레이아웃: 보행 동선을 따라 포토존과 휴식 공간 배치.
타입: 손글씨 스타일의 따뜻한 글씨체. **& 기호는 반드시 ‘&’ 그대로 사용**.
**포스터에 반드시 들어갈 정확한 텍스트(오탈자 없이 그대로):**
H1: **2024년 양림 & 크리스마스 문화축제**
H2: **2024. 12. 2.(월) ~ 12. 31.(화)**
H3: **양림동 일원 (양림오거리 ~ 선교묘역)**
Caption(하단): **주최/주관: 광주광역시 남구 / 양림 & 크리스마스 문화축제추진위원회**
슬로건(선택): **따뜻한 연말, 양림에서 만나다**
품질: 친근함, 아늑함, 문화적 소통, 공동체 참여.
Negative: 차가운 이미지, 혼잡함, 소외감.

==================== Minimal Graphic ====================
미니멀 그래픽 포스터, Aspect: 4:5, 최소 2160x2700 px, **정확한 한국어 텍스트**.
스타일: 팔레트(따뜻한 빨강, 녹색, 금색, 흰색)에서 2–3색 중심, 여백과 대비를 살린 클린 레이아웃.
레이아웃: 보행 동선을 따라 포토존과 휴식 공간 배치.
타입: 손글씨 스타일의 따뜻한 글씨체. **&는 기호 그대로**.
**정확히 표기할 텍스트:**
H1: **2024년 양림 & 크리스마스 문화축제**
H2: **2024. 12. 2.(월) ~ 12. 31.(화)**
H3: **양림동 일원 (양림오거리 ~ 선교묘역)**
Cap